### En este notebook crearemos los pattern para identficar los antecedentes

Carga de todas las librerías necesarias, utilizaremos Spacy para todo el procesamiento NLP

In [1]:
import json
from glob import glob
import spacy
from spacy.lang.es import Spanish
from spacy.pipeline import EntityRuler
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
from medspacy.target_matcher import TargetMatcher, TargetRule

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)

Función para aplicar los pattern que identifican las enfermedades en los informes train y guardarlos en el modelo entityRuler_antecedente_ner

In [2]:
def generate_rules(patterns):
    nlp = Spanish()
    ruler = EntityRuler(nlp, overwrite_ents=True, validate=True)
    ruler.add_patterns(patterns)
    nlp.add_pipe(ruler)
    nlp.to_disk("./models/entityRuler_antecedente_ner")

Creamos el archivo antecedentes_patterns_CIE10.jsonl donde almacenamos los patterns (mas 500) para detectar las enfermedades indentificadas en el conjunto train.
No nos interesa detectar el 100% de las enfermedades de estos informes, sólo los que están presentes en el archivo [Codigos_CIE-10_Comorbilidades_FUENCOVID.txt](./data/Codigos_CIE-10_Comorbilidades_FUENCOVID.txt).

In [3]:
with open("./data/antecedentes_patterns_CIE10.jsonl", encoding="utf8") as f:
    patterns = json.loads(f.read())
generate_rules(patterns)

Función para generar las reglas con el formato de mspacy desde los patterns creados que será utilizadas para asignar el código CEI10 a los antecedentes encontrados

In [4]:
def pattern2rule(pattern):
    # Comprobamos si el pattern es una cadena o un pattern de spaCy 
    if isinstance(pattern["pattern"], str):
        literal = pattern["pattern"]
        pattern_arg = None
    else:
        literal = str(pattern["pattern"])
        pattern_arg = pattern["pattern"]
    category = pattern["label"]
    attributes = pattern["attributes"]
    return TargetRule(literal, category, pattern_arg, attributes=attributes) 

rules = [pattern2rule(pattern) for pattern in patterns]
 
TargetRule.to_json(rules, "./data/cie10_rules_mspacy.jsonl")

Utilizaremos el archivo test.txt para ir testeando las reglas 

In [5]:
with open ("./data/test.txt", "r", encoding="utf-8") as f:
    informe_urgencias = f.read().split("\n")
informe_urgencias_test = str(informe_urgencias)
#print (informe_urgencias_test) 

Utilizado el modelo que hemos creado testeamos que las reglas identifican correctamente los antecedentes

In [6]:
nlp = spacy.load("./models/entityRuler_antecedente_ner")


doc = nlp(informe_urgencias_test)
antecedentes_urgencias = [ent.text for ent in doc.ents]

print (f"Número de antecedente en el informe de urgencias: {len(antecedentes_urgencias)}\n")
print (f"Antecedentes en el informe de urgencias: {antecedentes_urgencias}\n\n")    

# Visualizamos los antecedentes encontrados en el informe
colors = {"ANTEC": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
options = {"ents": ["ANTEC"], "colors": colors}
displacy.render(doc, style="ent", options=options)

Número de antecedente en el informe de urgencias: 18

Antecedentes en el informe de urgencias: ['Exfumadora', 'VIH', 'VIH positivo', 'HIV', 'virus de la inmunodeficiencia humana', 'VIH C3', 'inmunodeficiencia', 'polimialgia reumática', 'enfermedad de Forestier', 'embarazada 18 semanas', 'gestación de 38 semanas', '38 semanas de gestación', '20 semanas de gestación', 'gestación de 24 semanas', 'semana 35 de gestación', '39 semanas de gestación', 'Diabetes Mellitus', 'diabetes mellitus tipo 2']


